
# Linear Relaxation of Sigmoid

this notebook is a visualization tests to assess that the linear relaxation of relu is well encoded

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Activation("relu", input_dim=1))

In [ ]:
import sys

sys.path.append("../..")
from decomon.models import clone

In [ ]:
decomon_model = clone(model, method="hybrid")

In [ ]:
def relu(x):
    return np.maximum(0.0, x)

In [ ]:
lower = -10
upper = 10
x_min = lower * np.ones((1, 1))
x_max = upper * np.ones((1, 1))
box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

output = decomon_model.predict([box])

In [ ]:
x, u, w_u, b_u, l, w_l, b_l = output

In [ ]:
b_u

In [ ]:
decomon_model.inputs

In [ ]:
def linear_hull_upper_decomon(lower, upper):
    x_min = lower * np.ones((1, 1))
    x_max = upper * np.ones((1, 1))
    box = np.concatenate([x_min[:, None], x_max[:, None]], 1)
    # print(box)
    output = decomon_model.predict([box])
    _, _, w_u, b_u = output[:4]
    print(w_u, b_u)

    upper = np.maximum(0.0, w_u) * upper + np.minimum(0.0, w_u) * lower + b_u

    return upper.min(), w_u.min(), b_u.min()


def linear_hull_lower_decomon(lower, upper):
    x_min = lower * np.ones((1, 1))
    x_max = upper * np.ones((1, 1))
    box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

    output = decomon_model.predict([box])
    w_l, b_l = output[-2:]
    lower = np.maximum(0.0, w_l) * lower + np.minimum(0.0, w_l) * upper + b_l
    return lower.min(), w_l.min(), b_l.min()

In [ ]:
def func(x_min, x_max):
    n_step = 100
    if x_max < x_min:
        x_max_ = x_max + 0
        x_max = x_min
        x_min = x_max_
    x = np.linspace(x_min, x_max, n_step)
    y = np.linspace(-10, 10, 1000)

    _, w_u, b_u = linear_hull_upper_decomon(x_min, x_max)
    _, w_l, b_l = linear_hull_lower_decomon(x_min, x_max)
    s_x = relu(x)
    s_y = relu(y)
    z_x = w_u * x + b_u
    q_x = w_l * x + b_l

    plt.ylim(-2, 11)

    plt.plot(x, s_x, c="blue")
    plt.plot(x, z_x, c="orange")
    plt.plot(x, q_x, c="green")

    plt.plot(y, s_y, "--", c="k")
    plt.title("Linear Relaxation of ReLU")

In [ ]:
interact(
    func,
    x_min=widgets.FloatSlider(value=-10, min=-10, max=10.0, step=0.01, continuous_update=False),
    x_max=widgets.FloatSlider(value=10, min=-10, max=10, step=0.01, continuous_update=False),
)